# Chargement des données

In [115]:
import pandas as pd

Décommentez la cellule suivante si vous avez besoin d'installer la librairie qui nous servira à la recherche de motifs fréquents.

In [142]:
# Installation de la librairie permettant la recherche de motifs fréquents 
#! pip install mlxtend

In [143]:
from mlxtend.frequent_patterns import apriori
from mlxtend.frequent_patterns import association_rules

In [116]:
df = pd.read_csv("./data_pretraitees_v2.csv")

In [117]:
df.drop(columns=['coordonnee_x', 'coordonnee_y'], inplace=True)

In [118]:
print(df.columns)
print(df.shape)

Index(['plante_sc', 'plante_fr', 'plante_precision', 'grande_culture',
       'nebulosite', 'temperature', 'vent', 'insecte_sc', 'insecte_fr',
       'insecte_denominationPlusPrecise', 'insecte_CdNomtaxref',
       'bord de l'eau', 'bord de route', 'forêt', 'grande(s) culture(s)',
       'jardin privé', 'littoral', 'parc ou jardin public', 'prairie',
       'péri-urbain', 'rochers', 'rural', 'urbain'],
      dtype='object')
(675712, 23)


# Taxons de référence : plante_fr et insecte_sc

## Taxon Plantes

In [119]:
print("Taxon 'plante_precision' : ", df['plante_precision'].nunique())
print("Taxon 'plante_fr' : ", df['plante_fr'].nunique())
print("Taxon 'plante_sc' : ", df['plante_sc'].nunique())

Taxon 'plante_precision' :  7634
Taxon 'plante_fr' :  2216
Taxon 'plante_sc' :  2651


Dans un premier temps, on décide de travailler avec le taxon plante_fr qui semble être le plus général. Comme on cherche à faire ressortir notamment des associations insectes-plantes, on supprime les plantes sous-représentés pour la taxonomie plante_fr. On supprime alors les genres qui ont moins de 100 observations pour la taxonomie plante_fr.

In [120]:
df1=df.copy()

In [121]:
# Suppression des 2 autres taxons de plantes
df1.drop(columns=['plante_precision', 'plante_sc'], inplace=True)

In [122]:
df1.columns

Index(['plante_fr', 'grande_culture', 'nebulosite', 'temperature', 'vent',
       'insecte_sc', 'insecte_fr', 'insecte_denominationPlusPrecise',
       'insecte_CdNomtaxref', 'bord de l'eau', 'bord de route', 'forêt',
       'grande(s) culture(s)', 'jardin privé', 'littoral',
       'parc ou jardin public', 'prairie', 'péri-urbain', 'rochers', 'rural',
       'urbain'],
      dtype='object')

In [123]:
df1['plante_fr'].value_counts()

Le Lierre grimpant                   19704
Carotte sauvage                      15016
Achillée millefeuille                 9703
Berce des prés                        8821
Le Buddléia de David                  7566
                                     ...  
Néottie nid d'oiseau                     1
Orpin à feuilles spatulées               1
Centaurée de Lyon                        1
Ophrys brun                              1
Cynoglosse à feuilles de Giroflée        1
Name: plante_fr, Length: 2216, dtype: int64

In [124]:
# Comptage des éléments des colonnes 
plante_fr_counts = df1['plante_fr'].value_counts()

# Liste des éléments qui apparaissent au moins 100 fois
plante_fr_100_plus = plante_fr_counts[plante_fr_counts >= 100].index

#df=df.where(df['plante_sc'].isin(plante_sc_100_plus)).dropna(subset=['plante_sc'])

df1.drop(df1[~df1['plante_fr'].isin(plante_fr_100_plus)].index, inplace=True)

## Taxon Insectes

In [87]:
print("Nombre de valeurs différentes pour les taxons suivants :")
print("Taxon 'insecte_denominationPlusPrecise' : ", df1['insecte_denominationPlusPrecise'].nunique())
print("Taxon 'insecte_fr' : ", df1['insecte_fr'].nunique())
print("Taxon 'insecte_sc' : ", df1['insecte_sc'].nunique())

Nombre de valeurs différentes pour les taxons suivants :
Taxon 'insecte_denominationPlusPrecise' :  1359
Taxon 'insecte_fr' :  559
Taxon 'insecte_sc' :  500


On décide également de prendre comme taxon de référence insecte_sc dans un premier temps, afin de voir si l'on peut dégager des motifs et associations à un niveau plutôt général de taxonomie.

In [125]:
# On ne retient que les insectes pour lesquels on a un certain nombre d'observations. 

# Comptage des éléments  
insecte_sc_counts = df1['insecte_sc'].value_counts()

# Liste des éléments qui apparaissent au moins 100 fois
insecte_sc_100_plus = insecte_sc_counts[insecte_sc_counts >= 100].index

#df=df.where(df['insecte_sc'].isin(insecte_sc_100_plus)).dropna(subset=['insecte_sc'])

df1_sc=df1.copy()
df1_sc.drop(df1[~df1['insecte_sc'].isin(insecte_sc_100_plus)].index, inplace=True)

In [140]:
# Suppression des autres taxons insectes
df1_sc.drop(columns=['insecte_fr', 'insecte_denominationPlusPrecise',
       'insecte_CdNomtaxref'], inplace=True)

In [126]:
df1_sc['insecte_sc'].value_counts()

Bombus                              15468
Apis mellifera                      14292
Halictus, Lasioglossum et autres    13645
Eristalis                           11562
Neomyia, Calliphora et autres        9279
                                    ...  
Hemipenthes morio                     108
Melitaea phoebe                       105
Dinoptera collaris                    104
Trichopoda pennipes                   104
Sapigidae                             100
Name: insecte_sc, Length: 252, dtype: int64

## Recherche de motifs fréquents 

### Un exemple : MF pour le taxon 'insecte_sc' Bombus

In [144]:
df1_sc['insecte_sc'].value_counts()

Bombus                              15468
Apis mellifera                      14292
Halictus, Lasioglossum et autres    13645
Eristalis                           11562
Neomyia, Calliphora et autres        9279
                                    ...  
Hemipenthes morio                     108
Melitaea phoebe                       105
Dinoptera collaris                    104
Trichopoda pennipes                   104
Sapigidae                             100
Name: insecte_sc, Length: 252, dtype: int64

In [145]:
data_bombus=df1_sc.where(df1_sc['insecte_sc']=='Bombus').dropna(subset=['insecte_sc'])

In [146]:
# One hot encoding
df_encoded = pd.get_dummies(data_bombus)
df_encoded = df_encoded.astype(bool)

In [147]:
# Recherche des motifs fréquents
frequent_itemsets = apriori(df_encoded, min_support=0.5, use_colnames=True)

# Filtrage des motifs fréquents de taille minimale égale à 2
frequent_itemsets['length'] = frequent_itemsets['itemsets'].apply(lambda x: len(x))
frequent_itemsets = frequent_itemsets[frequent_itemsets['length'] >= 2].drop(columns='length')

# Classement par ordre décroissant de support 
frequent_itemsets = frequent_itemsets.sort_values(by='support', ascending=False)
frequent_itemsets

,support,itemsets
4,0.738169,"(insecte_sc_Bombus, grande_culture_Non)"
5,0.636346,"(insecte_sc_Bombus, nebulosite_0-25%)"
6,0.603181,"(insecte_sc_Bombus, temperature_20-30ºC)"


In [148]:
frequent_itemsets['itemsets'].unique()

array([frozenset({'insecte_sc_Bombus', 'grande_culture_Non'}),
       frozenset({'insecte_sc_Bombus', 'nebulosite_0-25%'}),
       frozenset({'insecte_sc_Bombus', 'temperature_20-30ºC'})],
      dtype=object)

In [149]:
data_bombus['nebulosite'].value_counts()

0-25%      9843
25-50%     2415
50-75%     1715
75-100%    1495
Name: nebulosite, dtype: int64

In [150]:
data_bombus['temperature'].value_counts()

20-30ºC    9330
10-20ºC    4952
>30°C       983
<10°C       203
Name: temperature, dtype: int64

La présence de Bombus d'une part avec une nébulosité de 0-25% et d'autre part avec une température de 20-30% ou une grande_culture sont des motifs fréquents. Toutefois, ces motifs n'apportent pas beaucoup d'informations nouvelles. Il n'est pas étonnant que des bombus soient visibles lors de beau temps. La fréquence des motifs est également renforcée par le fait que une nebulosité basse et la fourchette de température 20-30° sont celles les plus représentées dans le dataset. Par ailleurs, les bénévoles sont plus enclin à se livrer à l'exercice du SPIPOLL par beau temps : moins de personne ont pour passe-temps d'aller photographier des insectes sur des plantes lorsqu'il pleut ou que le ciel est couvert...

In [151]:
# Recherche de règles d'associations importantes 

rules = association_rules(frequent_itemsets, metric="lift", min_threshold=1.0, support_only=True)
rules

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction,zhangs_metric


Aucune règle d'association importante.

Choix de la métrique du lift pour évaluer l'importance de la règle d'association car : 
- échantillon de données grand pour le bombus 
- mesure la force de la dépendance entre deux éléments dans une règle d'association. Plus précisément, le lift compare la probabilité que les deux éléments apparaissent ensemble dans une transaction par rapport à la probabilité attendue s'ils étaient indépendants.

### Avec les 15 insectes les plus fréquents dans le dataset 

In [152]:
top_15_insects= df1_sc['insecte_sc'].value_counts().nlargest(15)
list_of_top_insects = top_15_insects.index.tolist()

In [154]:
for insect in list_of_top_insects : 

    data_insect=df1_sc.where(df1_sc['insecte_sc']==insect).dropna(subset=['insecte_sc'])

    # One hot encoding
    df_encoded = pd.get_dummies(data_insect)
    df_encoded = df_encoded.astype(bool)
    
    frequent_itemsets = apriori(df_encoded, min_support=0.5, use_colnames=True)

    # Filtrer les motifs fréquents pour une taille minimale de 2
    frequent_itemsets['length'] = frequent_itemsets['itemsets'].apply(lambda x: len(x))
    frequent_itemsets = frequent_itemsets[frequent_itemsets['length'] >= 2].drop(columns='length')

    # Classement par ordre décroissant de support 
    frequent_itemsets = frequent_itemsets.sort_values(by='support', ascending=False)
    print(insect, " : ")
    #print(frequent_itemsets)
    rules = association_rules(frequent_itemsets, metric="lift", min_threshold=1, support_only=True)
    print("Nombres de règles d'association intéressantes trouvées : ", len(rules))
    if len(rules)>0 : 
        print(rules)

Bombus  : 
Nombres de règles d'association intéressantes trouvées :  0
Apis mellifera  : 
Nombres de règles d'association intéressantes trouvées :  0
Halictus, Lasioglossum et autres  : 
Nombres de règles d'association intéressantes trouvées :  0
Eristalis  : 
Nombres de règles d'association intéressantes trouvées :  0
Neomyia, Calliphora et autres  : 
Nombres de règles d'association intéressantes trouvées :  0
Halictidae  : 
Nombres de règles d'association intéressantes trouvées :  0
Formicinae  : 
Nombres de règles d'association intéressantes trouvées :  0
Ichneumonidae  : 
Nombres de règles d'association intéressantes trouvées :  0
Sphaerophoria  : 
Nombres de règles d'association intéressantes trouvées :  0
Oedemera  : 
Nombres de règles d'association intéressantes trouvées :  0
Miridae  : 
Nombres de règles d'association intéressantes trouvées :  0
Andrenidae  : 
Nombres de règles d'association intéressantes trouvées :  0
Episyrphus balteatus  : 
Nombres de règles d'association in

La plupart des motifs fréquents qu'on retrouve le plus souvent pour les 15 insectes ci-dessus inclut l'insecte en question et la température (la plupart du temps entre 20 et 30 degrés) et l'insecte et une nébulosite de 0-25% (support environ égal à 80% pour la plupart). On voit également des motifs fréquents (+50%) incluant l'insecte et la présence de grande_culture. 
En revanche, on ne trouve aucune règle d'association importante (avec un lift supérieur à 1).

### Recherche sur toute la base 

In [158]:
print("Nombre d'insectes (pour value_counts()>100) dans la base pour les taxons choisis : ", df1_sc['insecte_sc'].nunique())

Nombre d'insectes (pour value_counts()>100) dans la base pour les taxons choisis :  252


In [161]:
for insect in df1_sc['insecte_sc'].unique().tolist() : 

    data_insect=df1_sc.where(df1_sc['insecte_sc']==insect).dropna(subset=['insecte_sc'])

    # One hot encoding
    df_encoded = pd.get_dummies(data_insect)
    df_encoded = df_encoded.astype(bool)
    
    frequent_itemsets = apriori(df_encoded, min_support=0.5, use_colnames=True)

    # Filtrer les motifs fréquents pour une taille minimale de 2
    frequent_itemsets['length'] = frequent_itemsets['itemsets'].apply(lambda x: len(x))
    frequent_itemsets = frequent_itemsets[frequent_itemsets['length'] >= 2].drop(columns='length')

    # Classement par ordre décroissant de support 
    frequent_itemsets = frequent_itemsets.sort_values(by='support', ascending=False)
    
    # Recherche de règles d'association avec un lift supérieur à 1 
    rules = association_rules(frequent_itemsets, metric="lift", min_threshold=1, support_only=True)
    
    # Affichage des règles d'association intéressantes
    if len(rules)>0: 
        print(insect, " : ", rules)

Pas de règles intéressantes trouvées.

# Conclusion intermédiaire

Pas de motifs fréquents significatifs ou de règles d'association particulièrement intéressantes entre les espèces d'insectes (insectes_sc), de plantes (plantes_sc), de l'habitat et les conditions météorologiques.

# Approndissement : changement de granularité pour les taxons

In [209]:
df2=df.copy()

## Taxons Plante_precision et insecte_sc

On a travaillé jusque-là avec les taxons les plus généraux, à savoir plante_fr et insecte_sc.
A présent, on opte pour une taxonomie plus précise pour les plantes et on conserve d'abord le taxon généralise insecte_sc.

### Taxon plante

In [180]:
df['plante_precision'].nunique()

7634

In [195]:
df['plante_precision'].value_counts()

Grande marguerite (Leucanthemum vulgare)                   2529
.                                                          2398
Carotte sauvage (Daucus carota)                            2247
Berce des prés (Heracleum sphondylium)                     1820
Pissenlit (Taraxacum officinale)                           1474
                                                           ... 
Lavande d'Afghanistan                                         1
Menthe à feuille ronde                                        1
Les Dahlias (Dahlia Cav. sp.) Dahlia arborea imperialis       1
passiflore x belloti purple passion                           1
Heuchera sanguinea Engelm.                                    1
Name: plante_precision, Length: 7634, dtype: int64

In [210]:
# Comptage des éléments des colonnes 
plante_precision_counts = df2['plante_precision'].value_counts()

# Liste des éléments qui apparaissent au moins 100 fois
plante_precision_100_plus = plante_precision_counts[plante_precision_counts >= 100].index

#df=df.where(df['plante_precision'].isin(plante_precision_100_plus)).dropna(subset=['plante_precision'])
df2.drop(df2[~df2['plante_precision'].isin(plante_precision_100_plus)].index, inplace=True)

In [211]:
df2['plante_precision'].nunique()

270

In [212]:
df2.drop(columns=['plante_sc', 'plante_fr'], inplace=True)

### Taxon insecte

On conserve dans un premier temps le taxon insecte_sc.

In [186]:
print("Nombre de valeurs différentes pour les taxons suivants :")
print("Taxon 'insecte_denominationPlusPrecise' : ", df2['insecte_denominationPlusPrecise'].nunique())
print("Taxon 'insecte_fr' : ", df2['insecte_fr'].nunique())
print("Taxon 'insecte_sc' : ", df2['insecte_sc'].nunique())

Nombre de valeurs différentes pour les taxons suivants :
Taxon 'insecte_denominationPlusPrecise' :  749
Taxon 'insecte_fr' :  485
Taxon 'insecte_sc' :  432


In [187]:
# On ne retient que les insectes pour lesquels on a un certain nombre d'observations. 

# Comptage des éléments  
insecte_sc_counts = df2['insecte_sc'].value_counts()

# Liste des éléments qui apparaissent au moins 100 fois
insecte_sc_100_plus = insecte_sc_counts[insecte_sc_counts >= 100].index

#df=df.where(df['insecte_sc'].isin(insecte_sc_100_plus)).dropna(subset=['insecte_sc'])

df2_sc=df2.copy()
df2_sc.drop(df2[~df2['insecte_sc'].isin(insecte_sc_100_plus)].index, inplace=True)

In [189]:
df2_sc.drop(columns=['insecte_fr', 'insecte_denominationPlusPrecise',
       'insecte_CdNomtaxref'], inplace=True)

### Recherche de motifs fréquents sur tout le dataset

In [191]:
print("Nombre d'insectes (pour value_counts()>100) dans la base pour les taxons choisis : ", df2_sc['insecte_sc'].nunique())

Nombre d'insectes (pour value_counts()>100) dans la base pour les taxons choisis :  119


In [192]:
for insect in df2_sc['insecte_sc'].unique().tolist() : 

    data_insect=df2_sc.where(df2_sc['insecte_sc']==insect).dropna(subset=['insecte_sc'])

    # One hot encoding
    df_encoded = pd.get_dummies(data_insect)
    df_encoded = df_encoded.astype(bool)
    
    frequent_itemsets = apriori(df_encoded, min_support=0.5, use_colnames=True)

    # Filtrer les motifs fréquents pour une taille minimale de 2
    frequent_itemsets['length'] = frequent_itemsets['itemsets'].apply(lambda x: len(x))
    frequent_itemsets = frequent_itemsets[frequent_itemsets['length'] >= 2].drop(columns='length')

    # Classement par ordre décroissant de support 
    frequent_itemsets = frequent_itemsets.sort_values(by='support', ascending=False)
    
    # Recherche de règles d'association avec un lift supérieur à 1 
    rules = association_rules(frequent_itemsets, metric="lift", min_threshold=1, support_only=True)
    
    # Affichage des règles d'association intéressantes
    if len(rules)>0: 
        print(insect, " : ", rules)

Andrena  :              antecedents           consequents  antecedent support  \
0    (nebulosite_0-25%)  (insecte_sc_Andrena)                 NaN   
1  (insecte_sc_Andrena)    (nebulosite_0-25%)                 NaN   

   consequent support  support  confidence  lift  leverage  conviction  \
0                 NaN      1.0         NaN   NaN       NaN         NaN   
1                 NaN      1.0         NaN   NaN       NaN         NaN   

   zhangs_metric  
0            NaN  
1            NaN  
Gymnosoma  :                antecedents             consequents  antecedent support  \
0      (nebulosite_0-25%)  (insecte_sc_Gymnosoma)                 NaN   
1  (insecte_sc_Gymnosoma)      (nebulosite_0-25%)                 NaN   

   consequent support  support  confidence  lift  leverage  conviction  \
0                 NaN      1.0         NaN   NaN       NaN         NaN   
1                 NaN      1.0         NaN   NaN       NaN         NaN   

   zhangs_metric  
0            NaN  
1    

On obtient que des règles incluant la nébulosité, ce qui n'apporte pas de grande nouveauté...

## Taxons Plante_precision et insecte_denominationPlusPrecise

Cette fois-ci, on conserve le taxon "plante_precision" pour les genres de plantes mais on approfondit le niveau de granularité pour les insectes en choisissant la taxonomie la plus précise : insecte_denominationPlusPrecise.
Avant cela, on supprime à nouveau les plantes et insectes sous-représentés pour les taxonomies choisies. 

### Taxon insecte

In [217]:
print("Nombre de valeurs différentes pour les taxons suivants :")
print("Taxon 'insecte_denominationPlusPrecise' : ", df2['insecte_denominationPlusPrecise'].nunique())
print("Taxon 'insecte_fr' : ", df2['insecte_fr'].nunique())
print("Taxon 'insecte_sc' : ", df2['insecte_sc'].nunique())

Nombre de valeurs différentes pour les taxons suivants :
Taxon 'insecte_denominationPlusPrecise' :  749
Taxon 'insecte_fr' :  485
Taxon 'insecte_sc' :  432


In [224]:
# Comptage des éléments des colonnes 
insecte_precision_counts = df2['insecte_denominationPlusPrecise'].value_counts()

# Liste des éléments qui apparaissent au moins 100 fois

insecte_precision_100_plus = insecte_precision_counts[insecte_precision_counts >= 100].index.tolist()
insecte_precision_50_plus = insecte_precision_counts[insecte_precision_counts >= 50].index.tolist()
insecte_precision_30_plus = insecte_precision_counts[insecte_precision_counts >= 30].index.tolist()
print("Nombres d'insectes ayant plus de 100 observations : ", len(insecte_precision_100_plus))
print("Nombres d'insectes ayant plus de 50 observations : ",len(insecte_precision_50_plus))
print("Nombres d'insectes ayant plus de 30 observations : ",len(insecte_precision_30_plus))

Nombres d'insectes ayant plus de 100 observations :  11
Nombres d'insectes ayant plus de 50 observations :  30
Nombres d'insectes ayant plus de 30 observations :  58


In [225]:
# On ne retient que les insectes pour lesquels on a un certain nombre d'observations. 

# Comptage des éléments  
insecte_sc_counts = df2['insecte_denominationPlusPrecise'].value_counts()

# Liste des éléments qui apparaissent au moins 50 fois
insecte_sc__plus = insecte_sc_counts[insecte_sc_counts >= 100].index

#df=df.where(df['insecte_sc'].isin(insecte_sc_100_plus)).dropna(subset=['insecte_sc'])

df2_precis=df2.copy()
df2_precis.drop(df2[~df2['insecte_denominationPlusPrecise'].isin(insecte_sc__plus)].index, inplace=True)

In [226]:
df2_precis.drop(columns=['insecte_fr', 'insecte_sc', 'insecte_CdNomtaxref'], inplace=True)

### Recherche de motifs fréquents sur tout le dataset

In [227]:
print("Nombre d'insectes (pour value_counts()>100) dans la base pour les taxons choisis : ", df2_precis['insecte_denominationPlusPrecise'].nunique())

Nombre d'insectes (pour value_counts()>100) dans la base pour les taxons choisis :  11


In [228]:
for insect in df2_precis['insecte_denominationPlusPrecise'].unique().tolist() : 

    data_insect=df2_precis.where(df2_precis['insecte_denominationPlusPrecise']==insect).dropna(subset=['insecte_denominationPlusPrecise'])

    # One hot encoding
    df_encoded = pd.get_dummies(data_insect)
    df_encoded = df_encoded.astype(bool)
    
    frequent_itemsets = apriori(df_encoded, min_support=0.5, use_colnames=True)

    # Filtrer les motifs fréquents pour une taille minimale de 2
    frequent_itemsets['length'] = frequent_itemsets['itemsets'].apply(lambda x: len(x))
    frequent_itemsets = frequent_itemsets[frequent_itemsets['length'] >= 2].drop(columns='length')

    # Classement par ordre décroissant de support 
    frequent_itemsets = frequent_itemsets.sort_values(by='support', ascending=False)
    
    # Recherche de règles d'association avec un lift supérieur à 1 
    rules = association_rules(frequent_itemsets, metric="lift", min_threshold=1, support_only=True)
    
    # Affichage des règles d'association intéressantes
    if len(rules)>0: 
        print(insect, " : ", rules)

Musca autumnalis  :                                           antecedents  \
0                                 (nebulosite_0-25%)   
1  (insecte_denominationPlusPrecise_Musca autumna...   

                                         consequents  antecedent support  \
0  (insecte_denominationPlusPrecise_Musca autumna...                 NaN   
1                                 (nebulosite_0-25%)                 NaN   

   consequent support  support  confidence  lift  leverage  conviction  \
0                 NaN      1.0         NaN   NaN       NaN         NaN   
1                 NaN      1.0         NaN   NaN       NaN         NaN   

   zhangs_metric  
0            NaN  
1            NaN  


# Conclusion 

On obtient toujours les mêmes résultats même pour des niveaux de granularité/taxonomie différents (ici, plus précis). Pas de motifs fréquents autre que des associations insectes-nébulosité 0-25%, ce qui n'apportent pas grand chose à l'analyse. 

Rappel : 
- seuls les insectes ayant plus de 100 enregistrements ont été conservés
- seuil de support choisi pour les motifs : 50% 
- condition sur la métrique des règles d'associations : lift supérieur à 1.

# Suppression des conditions climatiques et abaissement du seuil de support

Dans cette partie, on relance une recherche de motifs fréquents après avoir supprimé les colonnes relatives aux conditions météorologiques : vent, nébulosité et température. On cherche encore à mettre en lumière des associations plantes-insectes en abaissant le seuil de support de 0.5 à 0.3.

In [10]:
from mlxtend.frequent_patterns import apriori
from mlxtend.frequent_patterns import association_rules

In [230]:
df = pd.read_csv("./data_pretraitees_v2.csv")

In [231]:
df.columns

Index(['plante_sc', 'plante_fr', 'plante_precision', 'grande_culture',
       'nebulosite', 'temperature', 'vent', 'insecte_sc', 'insecte_fr',
       'insecte_denominationPlusPrecise', 'insecte_CdNomtaxref',
       'coordonnee_x', 'coordonnee_y', 'bord de l'eau', 'bord de route',
       'forêt', 'grande(s) culture(s)', 'jardin privé', 'littoral',
       'parc ou jardin public', 'prairie', 'péri-urbain', 'rochers', 'rural',
       'urbain'],
      dtype='object')

In [232]:
df.drop(columns=['nebulosite', 'temperature', 'vent', 
                 'coordonnee_x', 'coordonnee_y'], inplace=True)

In [233]:
data_bombus=df.where(df['insecte_sc']=="Bombus").dropna(subset=['insecte_sc'])

# One hot encoding
df_encoded = pd.get_dummies(data_bombus)
df_encoded = df_encoded.astype(bool)
    
frequent_itemsets = apriori(df_encoded, min_support=0.3, use_colnames=True)

# Filtrer les motifs fréquents pour une taille minimale de 2
frequent_itemsets['length'] = frequent_itemsets['itemsets'].apply(lambda x: len(x))
frequent_itemsets = frequent_itemsets[frequent_itemsets['length'] >= 2].drop(columns='length')

# Classement par ordre décroissant de support 
frequent_itemsets = frequent_itemsets.sort_values(by='support', ascending=False)

frequent_itemsets

,support,itemsets
8,0.547136,"(insecte_sc_Bombus, grande_culture_Oui)"
6,0.449244,"(insecte_sc_Bombus, rural)"
9,0.402241,"(insecte_sc_Bombus, insecte_fr_Les Bourdons no..."
7,0.373762,"(insecte_sc_Bombus, grande_culture_Non)"
10,0.327105,"(insecte_sc_Bombus, insecte_fr_Les Bourdons à ..."


Pour voir directement s'il y a des motifs fréquents qui associe une plante et un insecte en particulier, on décide de supprimer toutes les colonnes relatives aux habitats et de ne conserver que celles concernant les genres de plantes et d'insectes. 

On supprime également les lignes dont la valeur est "Plante inconnue" et les lignes dont le compte de la plante renseignée est inférieur à 100 occurences.

In [251]:
df_plantes_insectes=df.drop(columns=['grande_culture', "bord de l'eau",
       'bord de route', 'forêt', 'grande(s) culture(s)', 'jardin privé',
       'littoral', 'parc ou jardin public', 'prairie', 'péri-urbain',
       'rochers', 'rural', 'urbain'])

In [252]:
# Supprimer les lignes avec "Plante inconnue" dans la colonne "plante_sc"
df_plantes_insectes = df_plantes_insectes[df_plantes_insectes['plante_sc'] != 'Plante inconnue']

In [253]:
# Compter le nombre d'occurrences de chaque valeur dans la colonne 'plante_sc'
value_counts = df_plantes_insectes['plante_sc'].value_counts()

# Créer un masque pour filtrer les lignes dont la valeur apparaît au moins 50 fois
masque = df_plantes_insectes['plante_sc'].map(value_counts) >= 100

# Appliquer le masque pour obtenir le DataFrame final
df_plantes_insectes = df_plantes_insectes[masque]

In [254]:
df_plantes_insectes['plante_sc'].value_counts()

Hedera helix                                                            19704
Les Composées à fleurs jaunes à port dressé (famille des Asteraceae)    16742
Daucus carota                                                           15016
Les Carottes (Daucus sp)                                                13311
Les Chardons et Cirses (des espèces des genre Cirsium et Carduus)       13121
                                                                        ...  
Euphorbia verrucosa                                                       102
Apium graveolens                                                          101
Rudbeckia hirta                                                           101
Hyacinthoides non-scripta                                                 100
Melissa officinalis                                                       100
Name: plante_sc, Length: 611, dtype: int64

In [255]:
df_plantes_insectes.columns

Index(['plante_sc', 'plante_fr', 'plante_precision', 'insecte_sc',
       'insecte_fr', 'insecte_denominationPlusPrecise', 'insecte_CdNomtaxref'],
      dtype='object')

In [256]:
df_plantes_insectes.drop(columns=['plante_fr', 'plante_precision',  
                                  'insecte_fr', 'insecte_denominationPlusPrecise', 'insecte_CdNomtaxref'], 
                         inplace=True)

## Recherche de motifs fréquents plante-insecte dans la base

En utilisant les taxons insecte_sc et plante_sc.

In [257]:
# Insectes ayant plus de 100 enregistrements dans la base 
insects_counts = df_plantes_insectes['insecte_sc'].value_counts()

# Filtre des genres d'insectes avec plus de 10 enregistrements
plus_de_100_insectes = insects_counts[insects_counts > 100].index.tolist()
print("Nombre d'insectes avec plus de 100 enregistrements dans la base : ", len(plus_de_100_insectes))

Nombre d'insectes avec plus de 100 enregistrements dans la base :  291


In [258]:
for insect in plus_de_100_insectes : 

    data_insect=df_plantes_insectes.where(df_plantes_insectes['insecte_sc']==insect).dropna(subset=['insecte_sc'])

    # One hot encoding
    df_encoded = pd.get_dummies(data_insect)
    df_encoded = df_encoded.astype(bool)
    
    frequent_itemsets = apriori(df_encoded, min_support=0.3, use_colnames=True)

    # Filtrer les motifs fréquents pour une taille minimale de 2
    frequent_itemsets['length'] = frequent_itemsets['itemsets'].apply(lambda x: len(x))
    frequent_itemsets = frequent_itemsets[frequent_itemsets['length'] >= 2].drop(columns='length')

    # Classement par ordre décroissant de support 
    frequent_itemsets = frequent_itemsets.sort_values(by='support', ascending=False)
    
    # Affichage des motifs fréquents
    if len(frequent_itemsets)>0:
        print(insect, " : ")
        print(frequent_itemsets)
        # Recherche de règles d'association avec un lift supérieur à 1 à partir des motifs fréquents relevés
        rules = association_rules(frequent_itemsets, metric="lift", min_threshold=1, support_only=True)
        # Affichage des règles d'association intéressantes
        if len(rules)>0: 
            print(insect, " : ", rules)

Vespula, Dolichovespula  : 
   support                                           itemsets
2  0.38177  (plante_sc_Hedera helix, insecte_sc_Vespula, D...
Colletes hederae  : 
    support                                           itemsets
2  0.992084  (insecte_sc_Colletes hederae, plante_sc_Hedera...
Vespa velutina  : 
    support                                           itemsets
2  0.462006  (plante_sc_Hedera helix, insecte_sc_Vespa velu...
Vespa crabro  : 
    support                                           itemsets
2  0.475232  (plante_sc_Hedera helix, insecte_sc_Vespa crabro)
Villa  : 
    support                                    itemsets
2  0.356322  (plante_sc_Hedera helix, insecte_sc_Villa)
Chrysolina americana  : 
    support                                           itemsets
2  0.503546  (insecte_sc_Chrysolina americana, plante_sc_Ro...
Mesembrina  : 
    support                                         itemsets
2  0.351351  (plante_sc_Hedera helix, insecte_sc_Mesembrina)
Hem